# Import Library

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


# Preprocessing


## memuat data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/capstone/fix_combined_dataset.csv')
dataset.head()

,Title,Ingredients,Steps,Loves
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4


### menghilangkan angka dalam bahan


In [ ]:
import re
dataset['Ingredients1'] = dataset['Ingredients'].astype(str).apply(lambda x: re.sub(r'\d+', '', x))
dataset['tambahan'] = dataset['Title'] + ' bahan ' + dataset['Ingredients1']
dataset

,Title,Ingredients,Steps,Loves,Ingredients1,tambahan
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,Ekor Ayam Kampung (potong )-- Buah Jeruk Nipi...,Ayam Woku Manado bahan Ekor Ayam Kampung (pot...
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,kg ayam (dipotong sesuai selera jangan kecil ...,Ayam goreng tulang lunak bahan kg ayam (dipot...
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/ kg ayam-- buah cabai hijau besar-- buah caba...,Ayam cabai kawin bahan / kg ayam-- buah cabai ...
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,gr daging ayam (saya pakai fillet)--Secukupny...,Ayam Geprek bahan gr daging ayam (saya pakai ...
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,gr kulit ayam & lemaknya-- siung bawang putih...,Minyak Ayam bahan gr kulit ayam & lemaknya-- ...
...,...,...,...,...,...,...
15588,Udang saus mentega,1 kg udang segar--1 buah bawang bombay kecil--...,"Bersihkan udang, siapkan bahan bahan--Tumis ba...",46,kg udang segar-- buah bawang bombay kecil-- s...,Udang saus mentega bahan kg udang segar-- bua...
15589,Jipang tumis udang,500 gr jipang(saya pk 6bh jipang muda yg kecil...,Tumis bawang merah bawang putih hingga harum.-...,6,"gr jipang(saya pk bh jipang muda yg kecil),po...",Jipang tumis udang bahan gr jipang(saya pk bh...
15590,Tumis Udang Tempe,3 siung bawang merah--3 siung bawang putih--2 ...,"Tiriskan bawang putih, bawang merah dan cabe l...",3,siung bawang merah-- siung bawang putih-- bua...,Tumis Udang Tempe bahan siung bawang merah-- ...
15591,Tumis kacang panjang terong udang,"Stgah ikat kacang panjang,ku bli 3ribu d psr--...","Potong2 kacang panjang,terong cuci bersih sisi...",12,"Stgah ikat kacang panjang,ku bli ribu d psr--T...",Tumis kacang panjang terong udang bahan Stgah ...


### meng-implementasikan stopword indonesia


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dataset['tambahan'] = dataset['tambahan'].apply(remove_stopwords)
dataset['tambahan'] = dataset['tambahan'].str.replace('gr', '')
dataset['tambahan'] = dataset['tambahan'].str.replace('kg', '')
dataset['tambahan'] = dataset['tambahan'].str.replace('&', '')


In [ ]:
dataset['tambahan']

0        Ayam Woku Manado bahan Ekor Ayam Kampung ( pot...
1        Ayam goreng tulang lunak bahan  ayam ( dipoton...
2        Ayam cabai kawin bahan /  ayam -- buah cabai h...
3        Ayam Geprek bahan  daging ayam ( pakai fillet ...
4        Minyak Ayam bahan  kulit ayam  lemaknya -- siu...
                               ...                        
15588    Udang saus mentega bahan  udang segar -- buah ...
15589    Jipang tumis udang bahan  jipang ( pk bh jipan...
15590    Tumis Udang Tempe bahan siung bawang merah -- ...
15591    Tumis kacang terong udang bahan Stgah ikat kac...
15592    Siomay Udang Ayam bahan bh dada ayam -- /  uda...
Name: tambahan, Length: 15593, dtype: object

In [ ]:
dataset['tambahan'][0]

'Ayam Woku Manado bahan Ekor Ayam Kampung ( potong ) -- Buah Jeruk Nipis -- Sdm Garam -- Ruas Kunyit -- Bawang Merah -- Bawang Putih -- Cabe Merah -- Cabe Rawit Merah ( sesuai selera ) -- Butir Kemiri -- Batang Sereh -- Lembar Daun Salam -- Ikat Daun Kemangi -- Penyedap -- / Gelas Air --'

spliting data


## Membuat matrix TFIDF

IDF digunakan untuk memberikan bobot yang lebih besar pada kata-kata yang jarang muncul dalam keseluruhan dataset. 

In [ ]:
# Inisialisasi objek CountVectorizer
vectorizer = CountVectorizer()

# Konversi teks menjadi matriks term frequency
tf_matrix = vectorizer.fit_transform(dataset['Title'])

# Menghitung inverse document frequency
# menghitung logaritma dari jumlah total resep dalam dataset dibagi dengan jumlah resep yang mengandung kata tersebut.
idf = np.log(tf_matrix.shape[0] / np.sum(tf_matrix > 0, axis=0))

# Mengalikan term frequency dengan inverse document frequency
# menghasilkan matriks TF-IDF, yang memperhitungkan frekuensi kata dalam resep serta bobot kata berdasarkan inverse document frequency.
tfidf_matrix = tf_matrix.multiply(idf)


## Menghitung similarity 


* Cosine similarity adalah metrik yang digunakan untuk mengukur kesamaan antara dua vektor. 

* menghitung kesamaan antara setiap pasangan vektor resep dalam matriks TF-IDF.

* Nilai cosine similarity berkisar antara -1 hingga 1

* Secara ringkas, cosine similarity menghitung kesamaan antara resep-resep menggunakan cosine similarity, dan hasilnya disimpan dalam similarity_matrix untuk digunakan dalam sistem rekomendasi.

In [ ]:
# Menghitung similarity antar resep menggunakan cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix)


In [ ]:
# print(similarity_matrix)[1]

spliting data


# Modelling

Mengimplementasikan model content-based filtering dengan menggunakan TensorFlow:

In [ ]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(similarity_matrix.shape[0], 100, input_length=1))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))  # Menambahkan lapisan Dropout
model.add(Dense(similarity_matrix.shape[0], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[Accuracy()])


In [ ]:
# from tensorflow.keras.metrics import Accuracy
# # Mendefinisikan model
# model = Sequential()
# model.add(Embedding(similarity_matrix.shape[0], 100, input_length=1))
# model.add(Flatten())
# model.add(Dense(similarity_matrix.shape[0], activation='softmax'))

# # Kompilasi model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[Accuracy()])

# # # Melatih model
# # model.fit(np.arange(similarity_matrix.shape[0]), similarity_matrix, epochs=10)
# # Melatih model dan mencatat riwayat pelatihan
# history = model.fit(np.arange(similarity_matrix.shape[0]), similarity_matrix, epochs=10)

# # Menampilkan akurasi di setiap epoch
# for epoch, acc in enumerate(history.history['accuracy']):
#     print(f'Epoch {epoch + 1}: Akurasi = {acc}')


In [ ]:
history = model.fit(np.arange(similarity_matrix.shape[0]), similarity_matrix, epochs=10)

# Menampilkan akurasi di setiap epoch
for epoch, acc in enumerate(history.history['accuracy']):
    print(f'Epoch {epoch + 1}: Akurasi = {acc}')

Epoch 1/10
488/488 [==============================] - 27s 50ms/step - loss: 2844.6494 - accuracy: 0.0000e+00
Epoch 2/10
488/488 [==============================] - 26s 53ms/step - loss: 5055.6157 - accuracy: 0.0132
Epoch 3/10
488/488 [==============================] - 26s 54ms/step - loss: 21236.3555 - accuracy: 0.3838
Epoch 4/10
488/488 [==============================] - 28s 57ms/step - loss: 73588.9062 - accuracy: 0.7582
Epoch 5/10
488/488 [==============================] - 25s 51ms/step - loss: 177642.4688 - accuracy: 0.8090
Epoch 6/10
488/488 [==============================] - 26s 54ms/step - loss: 344298.1875 - accuracy: 0.8189
Epoch 7/10
488/488 [==============================] - 26s 53ms/step - loss: 574890.3750 - accuracy: 0.8211
Epoch 8/10
488/488 [==============================] - 26s 53ms/step - loss: 879407.4375 - accuracy: 0.8219
Epoch 9/10
488/488 [==============================] - 24s 50ms/step - loss: 1250393.8750 - accuracy: 0.8214
Epoch 10/10
488/488 [=================

Membuat fungsi untuk merekomendasikan resep berdasarkan input pengguna:

In [ ]:
def recommend_recipe(recipe_name, k=10):
    # Mendapatkan indeks resep berdasarkan nama resep
    recipe_index = dataset[dataset['Title'] == recipe_name].index[0]

    # Mendapatkan similarity score antara resep tersebut dengan resep lainnya
    recipe_scores = model.predict(np.array([recipe_index]))[0]

    # Mengurutkan resep berdasarkan similarity score
    top_indices = np.argsort(recipe_scores)[::-1][1:k+1]

    # Mengembalikan daftar rekomendasi resep
    return dataset.loc[top_indices, 'Title']


In [ ]:
# Memanggil fungsi recommend_recipe untuk merekomendasikan resep berdasarkan input pengguna
recommendations = recommend_recipe('Tumis Udang Tempe')
print(recommendations)


1/1 [==============================] - 0s 113ms/step
13592                           Tahu Tempe Telur Balado
14430                           Balado tahu tempe udang
9359                       Ayam,tahu,tempe bumbu kuning
7717                           Tahu Goreng Telur Simple
12762                           Balado Tahu Tempe Telur
11337                           Tahu Tempe Telur Balado
9473                    Tahu tempe goreng sambal balado
10436                                Tempe Goreng Telur
9368                             Botok Udang Tahu Tempe
5766     👉 Sambal Goreng Hati sapi dan tempe 👈 #Ketopad
Name: Title, dtype: object


In [ ]:
dataset.to_csv('dataset.csv', index=False)
